In [37]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
%cd /content/drive/MyDrive/Colab Notebooks/COSE474 TP

/content/drive/MyDrive/Colab Notebooks/COSE474 TP


# Import

In [1]:
!pip install datasets
!pip install accelerate
!pip install evaluate

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/265.7 kB ? eta -:--:--
   --------------------------------------- 265.7/265.7 kB 16.0 MB/s eta 0:00:00


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/84.1 kB ? eta -:--:--
   ---------------------------------------- 84.1/84.1 kB ? eta 0:00:00


In [2]:
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm

from transformers import BertModel, BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

c:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
train_path = './train_low.csv'
test_path = './test_low_10000.csv'
dataset = load_dataset('csv', data_files={'train': train_path, 'test': test_path})

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 125.00it/s]
Generating train split: 2000 examples [00:00, 60597.76 examples/s]
Generating test split: 10000 examples [00:00, 101058.56 examples/s]


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()
del model

In [22]:
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=2).to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def tokenization(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenization, batched=True, remove_columns=["text"])
tokenized_dataset.set_format("torch")

Map: 100%|██████████| 20000/20000 [00:05<00:00, 3476.45 examples/s]


In [6]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
training_args = TrainingArguments(
    output_dir='./checkpoints/baseline',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    optim='adamw_torch',
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    seed=42,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch'

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['train'],
    compute_metrics=compute_metrics
)

In [24]:
trainer.train()

 10%|█         | 125/1250 [05:57<53:34,  2.86s/it] 

{'loss': 0.6912, 'learning_rate': 1.25e-05, 'epoch': 1.0}


                                                  
 10%|█         | 125/1250 [09:52<53:34,  2.86s/it]

{'eval_loss': 0.6675734519958496, 'eval_accuracy': 0.598, 'eval_runtime': 234.8294, 'eval_samples_per_second': 8.517, 'eval_steps_per_second': 0.532, 'epoch': 1.0}


 20%|██        | 250/1250 [15:50<47:33,  2.85s/it]   

...

                                                     
100%|██████████| 1250/1250 [1:50:42<00:00,  2.91s/it]

{'eval_loss': 0.4140821397304535, 'eval_accuracy': 0.7675, 'eval_runtime': 236.6089, 'eval_samples_per_second': 8.453, 'eval_steps_per_second': 0.528, 'epoch': 10.0}


100%|██████████| 1250/1250 [1:50:43<00:00,  5.31s/it]

{'train_runtime': 6643.6376, 'train_samples_per_second': 3.01, 'train_steps_per_second': 0.188, 'train_loss': 0.5671640991210938, 'epoch': 10.0}


TrainOutput(global_step=1250, training_loss=0.5671640991210938, metrics={'train_runtime': 6643.6376, 'train_samples_per_second': 3.01, 'train_steps_per_second': 0.188, 'train_loss': 0.5671640991210938, 'epoch': 10.0})

In [39]:
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy
1,136222588534.783997,nan,0.500000


{'eval_loss': nan, 'eval_accuracy': 0.5}

In [38]:
trainer.evaluate(eval_dataset=tokenized_dataset['test'])

  7%|▋         | 93/1250 [03:29<2:23:17,  7.43s/it]